# Sentinel Data

In [2]:
import mlstac

mlstac.download(
    snippet="isp-uv-es/CloudSEN12Plus", path="data", split="test", quiet=True
)  # use "all" to download the entire dataset
# Load the ML-STAC collection
ds = mlstac.load(snippet="data/main.json")

In [ ]:
subset = ds.metadata[
    (ds.metadata["split"] == "test")
    & (ds.metadata["label_type"] == "high")
    & (ds.metadata["proj_shape"] == 509)
][:10]
datacube = mlstac.get_data(dataset=subset, quiet=True)
datacube.shape

In [ ]:
# Plot the first datapoint

import matplotlib.pyplot as plt
import numpy as np

datapoint = datacube[0]
datapoint_rgb = np.moveaxis(datapoint[[3, 2, 1]], 0, -1) / 5000
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(datapoint_rgb)
ax[0].set_title("RGB")
ax[1].imshow(datapoint[13], cmap="gray")
ax[1].set_title("Human label")
ax[2].imshow(datapoint[14], cmap="gray")
ax[2].set_title("UnetMobV2 v1.0")
plt.show()

In [ ]:
datapoint = datacube[0]
datapoint_rgb = datapoint[[1, 2, 3, 7, 10], ...].astype(np.float32)  # BGRNIR
datapoint.shape, datapoint_rgb.shape

# Validation

In [1]:
import torch
import pytorch_lightning as pl
from utils.ghostnetv2 import ghostnetv2
from utils.test_utils import CoreDataModule, LitModel, load_fi_weights

torch.set_float32_matmul_precision("high")

In [2]:
# Define the datamodule
datamodule = CoreDataModule(batch_size=128)

# Define the trainer
trainer = pl.Trainer(
    max_epochs=100, callbacks=None, accelerator="auto", precision="16-mixed"
)

# Define the model
model = ghostnetv2(error_model="single", inject_p=0.0001, inject_epoch=0)
model = load_fi_weights(model, "weights/GN_SSL_280.pt")
x = torch.randn(1, 5, 512, 512)
y, intermediates = model(x)
print(y.shape)
print(intermediates[-1].shape)

model = LitModel(model=model)

# Start the training
_ = trainer.validate(model=model, datamodule=datamodule)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded 764 weights
torch.Size([1, 1000])
torch.Size([1, 1536, 16, 16])
Validation DataLoader 0: 100%|██████████| 5/5 [00:10<00:00,  0.49it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.5249253511428833
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# From TIF file to numpy array

import numpy as np
import rasterio

img_path = "data/2021-09-01T10-00-00Z.tif"

# Open the images using rasterio
with rasterio.open(img_path) as img:
    b02 = img.read(2)  # Band 2 = Blue
    b03 = img.read(3)  # Band 3 = Green
    b04 = img.read(4)  # Band 4 = Red
    b08 = img.read(8)  # Band 8 = NIR
    b11 = img.read(11)  # Band 11 = SWIR1

    # Stack the bands
    img_image = np.stack([b02, b03, b04, b08, b11], axis=0).astype(np.float32)